In [ ]:
!pip install Selenium

!apt-get update # to update ubuntu to correctly run apt install

!apt install chromium-chromedriver

     |████████████████████████████████| 979 kB 12.6 MB/s 
     |████████████████████████████████| 359 kB 64.6 MB/s 
     |████████████████████████████████| 138 kB 55.9 MB/s 
     |████████████████████████████████| 4.0 MB 77.1 MB/s 
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 58 kB 4.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:

place = '' # <- 크롤링할 지역 입력
post_list = []
kind_list = []


for i in range(1,3) :
    browser = webdriver.Chrome('chromedriver',options=options)
    browser.get('https://www.mangoplate.com/search/'+place+'?keyword='+place+'&page='+str(i))
    
    for i in range (1,11):
        for j in range (1,3) :
            ll = browser.find_element(By.XPATH, "//ul[@class='list-restaurants']/li["+str(i)+"]/div["+str(j)+"]/figure/figcaption/div/a").get_attribute('href')
            kind = browser.find_element(By.XPATH, "//ul[@class='list-restaurants']/li["+str(i)+"]/div["+str(j)+"]/figure/figcaption/div/p[1]/span").get_attribute('innerText')           
            post_list.append(ll)
            kind_list.append(kind)

In [ ]:
review=[] # 리뷰
taste = [] # 맛있다/괜찮다/별로
title = [] # 식당 이름
t_1 = [] # 가고싶다 수
t_2 = [] # 전체 평점
locate = [] # 주소
kind = [] # 음식종류


for k in range (len(post_list)) : 
    wd = webdriver.Chrome('chromedriver',options=options)
    wd.get(post_list[k])
    N = wd.find_element(By.XPATH,"/html/body/main/article/div[1]/div[1]/div/section[3]/header/ul/li[1]/button/span").text

    if int(N)<=5:
        req = wd.page_source
        soup = BeautifulSoup(req, 'html.parser')
        aa = soup.find_all('p',"RestaurantReviewItem__ReviewText") # 리뷰
        bb = soup.find_all('span',"RestaurantReviewItem__RatingText") # 맛있다/괜찮다/별로
        cc = soup.find('h1','restaurant_name').text # 식당 이름
        dd = wd.find_element(By.XPATH, "/html/body/main/article/div[1]/div[1]/div/section[1]/header/div[2]/span[3]").text # 가고싶다 수
        try:
          ee = soup.find('strong','rate-point').text # 전체 평점
        except :
          ee = '평점없음' 
        ff = soup.find('span','Restaurant__InfoAddress--Text').text # 주소
        gg = kind_list[k] # 음식종류
        
        
        for i in range(len(aa)) :
            review.append(aa[i].text)
            taste.append(bb[i].text)
            title.append(cc)
            t_1.append(dd)
            t_2.append(ee)
            locate.append(ff)
            kind.append(gg)
            
            
    if int(N)%5 == 0 :
        N1 = int(N)//5-1
    else :
        N1 = int(N)//5

    for i in range(N1):
        element = wd.find_element(By.XPATH, "/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]")
        wd.execute_script("arguments[0].click();", element)
        time.sleep(2)
        
    req = wd.page_source
    soup = BeautifulSoup(req, 'html.parser')
    aa = soup.find_all('p',"RestaurantReviewItem__ReviewText") # 리뷰
    bb = soup.find_all('span',"RestaurantReviewItem__RatingText") # 맛있다/괜찮다/별로
    cc = soup.find('h1','restaurant_name').text # 식당 이름
    dd = wd.find_element(By.XPATH, "/html/body/main/article/div[1]/div[1]/div/section[1]/header/div[2]/span[3]").text # 가고싶다 수
    try:
        ee = soup.find('strong','rate-point').text # 전체 평점
    except :
        ee = '평점없음'  
    ff = soup.find('span','Restaurant__InfoAddress--Text').text # 주소
    gg= kind_list[k] # 음식종류
    
        
    for i in range(len(aa)) :
        review.append(aa[i].text)
        taste.append(bb[i].text)
        title.append(cc)
        t_1.append(dd)
        t_2.append(ee)
        locate.append(ff)
        kind.append(gg)
        
        
        
   
        
TOTAL = pd.DataFrame({'review':review, 'taste':taste, 'title':title, '가고싶다':t_1, '전체평점':t_2, '주소':locate, '음식종류':kind})

In [ ]:
TOTAL.to_csv() # csv저장할 주소 입력